# Statistical Testing

Research Question Documentation: "Does the new content loading system significantly reduce the average app startup time compared to our current system?"

Key Components Defined:

Change: New content loading system
Metric: App startup time (measured in milliseconds)
Population: Active app users
Direction: Looking for a reduction in time (less than)
Formulate Hypotheses: Since we specifically want to test for improvement (reduction in time), this calls for a one-sided test.

H₀ (null): μnew >= μcurrent (New system is not faster than current system)
H₁ (alternative): μnew < μcurrent (New system is faster than current system)

In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Define data collection parameters
collection_plan = {
    'metrics': ['startup_time_ms', 'device_type', 'app_version', 'network_type'],
    'start_date': datetime.now(),
    'duration_days': 14,
    'control_variables': ['app_version', 'network_type'],
    'exclusion_criteria': ['startup_time_ms > 10000', 'network_type == "offline"']
}

# Stratify Sampling
def analyze_device_distribution(data_df):
    """
    Analyzes the current distribution of device types in our user base
    Returns the proportion of each device type
    """
    device_counts = data_df['device_type'].value_counts()
    total_users = len(data_df)
    device_proportions = device_counts / total_users
    
    print("Current device distribution:")
    for device, proportion in device_proportions.items():
        print(f"{device}: {proportion:.1%}")
    
    return device_proportions

def stratified_sample_allocation(user_df, group_size, device_proportions):
    """
    Allocates users to groups while maintaining device type proportions
    
    Parameters:
    user_df: DataFrame with user information
    group_size: Desired size of each group
    device_proportions: Target proportions for each device type
    
    Returns: Dictionary with user IDs for each group
    """
    control_group = []
    treatment_group = []
    
    # Calculate target number of users for each device type
    target_counts = {
        device: int(group_size * proportion)
        for device, proportion in device_proportions.items()
    }
    
    # Sample users for each device type
    for device_type in target_counts:
        device_users = user_df[user_df['device_type'] == device_type]
        
        # Randomly sample users of this device type
        sampled_users = device_users.sample(
            n=target_counts[device_type] * 2  # Times 2 for both groups
        )
        
        # Split sampled users between groups
        mid_point = len(sampled_users) // 2
        control_group.extend(sampled_users.iloc[:mid_point].index)
        treatment_group.extend(sampled_users.iloc[mid_point:].index)
    
    return {
        'control': control_group,
        'treatment': treatment_group
    }

def collect_startup_time(user_id, group):
    """
    Collects startup time data for a single user session
    Now includes stratification information
    """
    return {
        'user_id': user_id,
        'group': group,
        'startup_time_ms': None,  # To be filled with actual measurement
        'timestamp': datetime.now(),
        'device_type': None,      # To be filled
        'app_version': None       # To be filled
    }

# Example usage:
def run_stratified_experiment(data_df, group_size=1000):
    """
    Sets up and runs a stratified experiment
    """
    # First, analyze current device distribution
    device_props = analyze_device_distribution(data_df)
    
    # Allocate users to groups maintaining device proportions
    groups = stratified_sample_allocation(data_df, group_size, device_props)
    
    # Verify stratification worked as expected
    print("\nVerifying group distributions:")
    for group_name, user_ids in groups.items():
        group_df = data_df.loc[user_ids]
        print(f"\n{group_name.title()} Group Device Distribution:")
        device_dist = group_df['device_type'].value_counts(normalize=True)
        for device, prop in device_dist.items():
            print(f"{device}: {prop:.1%}")
    
    return groups